# Обучение моделей нейронных сетей

## Задача

Обучение CNN, LSTM, LSTM_CNN моделей текстах обработанных при помощи Word2Vec и FastText.

## Данные

Классы с архитектурами нейронных сетей и данные для обучения и теста взяты из файла *dataset_and_models.py*,  
гиперпараметры для сетей записаны в файле *data/best_hyperparameters.pkl*

## Расчёты

In [1]:
import pickle
import torch
from torch import nn, optim
from sklearn.metrics import f1_score, accuracy_score
from dataset_and_models import CNN, LSTM, LSTM_CNN
from dataset_and_models import w2v_data_train, w2v_data_test
from dataset_and_models import fasttext_data_train, fasttext_data_test
from tqdm.notebook import tqdm

In [2]:
# чтение файла, содержащего лучшие гиперпараметры
with open('data/best_hyperparameters.pkl', 'rb') as f:
    best_hyperparameters = pickle.load(f)

In [3]:
# установка устройва для расчётов
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
def train(
        model_class, 
        hyperparameters, 
        data_train, 
        data_test, 
        indicators_switch=True, 
        epochs=30, 
        device=DEVICE, 
        name_model=None
):
    """
    Функция для обучения модели
    :param model_class: класс модели нейронной сети
    :param hyperparameters: гиперпараметры модели
    :param data_train: данные для обучения
    :param data_test: данные для теста
    :param indicator_switch: отображать информацию об обучение каждую эпоху или нет
    :param epochs: количество эпох обучения
    :param device: устройво для основных расчётов
    :param name_model: имя подели для сохранения
    """
    # получение коэффициента скорости обучения
    lr = hyperparameters.pop('lr')
    # создание объекта модели нейронной сети
    model = model_class(**hyperparameters, p=0.25)
    
    # установка функции ошибки и оптимизатора
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # установка планировщика, для умножения lr на коэффициент 0.5 если F1-score не будет улучшаться в течении 3 эпох
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max', 
        factor=0.5,
        patience=2
    )

    # хранит лучший F1-score
    best_f1_score = 0

    # цикл обучения
    for epoch in range(epochs):

        model.to(device)
        model.train()

        progress_bar = tqdm(data_train, desc=f'Эпоха {epoch+1}') if indicators_switch else data_train

        # обучение модели в пределах одной эпохи
        for x, y in progress_bar:

            x, y = x.to(device), y.to(device)
            y_hat = model(x)

            optimizer.zero_grad()
            loss = loss_fn(y_hat, y)
            loss.backward()
            optimizer.step()

        # оценка модели после эпохи обучения    
        model.eval()
        
        output_model = torch.Tensor([]).to(device)
        output_y = torch.Tensor([]).to(device)
        
        with torch.no_grad():
            for x, y in data_test:
                x, y = x.to(device), y.to(device)
                output_model = torch.concat([output_model, model(x).argmax(axis=1)])
                    
                output_y = torch.concat([output_y, y])
        
        output_y = output_y.to('cpu')
        output_model = output_model.to('cpu')
        f1 = f1_score(y_true=output_y.numpy(),
                      y_pred=output_model.numpy(),
                      average='weighted')
        
        scheduler.step(f1)

        # вывод информации об оценках модели по итогу эпохи
        if indicators_switch:

            print('F1-score (weighted):\t', f1)
            print('Точность:\t\t', accuracy_score(y_true=output_y.numpy(),
                                                  y_pred=output_model.numpy()))
        
        # сохранение модели с наилучшим F1-score
        if best_f1_score < f1:
            
            best_f1_score = f1
            torch.save(model, f'{name_model}.pt')

            if indicators_switch:
                print('===Модель сохранена===')

### Обучение CNN моделей

In [5]:
# w2v CNN
train(
    model_class=CNN,
    hyperparameters=best_hyperparameters['w2v_CNN'].copy(),
    data_train=w2v_data_train,
    data_test=w2v_data_test,
    name_model='models/w2v_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.669712503722155
Точность:		 0.7247386759581882
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7219501993229656
Точность:		 0.7456445993031359
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7164233318899298
Точность:		 0.7202295552367288


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7152114489616587
Точность:		 0.7202295552367288


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7154793378554313
Точность:		 0.7202295552367288


In [6]:
# fasttext CNN
train(
    model_class=CNN,
    hyperparameters=best_hyperparameters['fasttext_CNN'].copy(),
    data_train=fasttext_data_train,
    data_test=fasttext_data_test,
    name_model='models/fasttext_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6067739349233938
Точность:		 0.6870260299241648
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6640789982666838
Точность:		 0.7286329165812666
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7275976482823444
Точность:		 0.7323221971715516


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7280868925988653
Точность:		 0.7329370772699324


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7276969259354683
Точность:		 0.7319122771059643


### Обучение LSTM моделей

In [7]:
# w2v LSTM
train(
    model_class=LSTM,
    hyperparameters=best_hyperparameters['w2v_LSTM'].copy(),
    data_train=w2v_data_train,
    data_test=w2v_data_test,
    name_model='models/w2v_lstm'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6019555431348209
Точность:		 0.6894855503176881
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.643226501963163
Точность:		 0.7230989956958394
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7202674583163496
Точность:		 0.7255585160893626


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7213643773721308
Точность:		 0.7267882762861242


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7198570380873437
Точность:		 0.7245337159253945


In [8]:
# fasttext LSTM
train(
    model_class=LSTM,
    hyperparameters=best_hyperparameters['fasttext_LSTM'].copy(),
    data_train=fasttext_data_train,
    data_test=fasttext_data_test,
    name_model='models/fasttext_lstm'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.4426738226189907
Точность:		 0.593974175035868
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6292840179528574
Точность:		 0.7073170731707317
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7100359512161554
Точность:		 0.7345767575322812


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7109631697674313
Точность:		 0.7349866775978684


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7206272332854265
Точность:		 0.7333469973355196


### Обучение LSTM_CNN моделей

In [9]:
# w2v LSTM_CNN
train(
    model_class=LSTM_CNN,
    hyperparameters=best_hyperparameters['w2v_LSTM_CNN'].copy(),
    data_train=w2v_data_train,
    data_test=w2v_data_test,
    name_model='models/w2v_lstm_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6813714266836717
Точность:		 0.7237138757942201
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6884577927954262
Точность:		 0.7302725968436155
===Модель сохранена===


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7440169396536029
Точность:		 0.7501537200245952


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7434516518084021
Точность:		 0.749743799959008


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7423593970574923
Точность:		 0.74871899979504


In [10]:
# fasttext LSTM_CNN
train(
    model_class=LSTM_CNN,
    hyperparameters=best_hyperparameters['fasttext_LSTM_CNN'].copy(),
    data_train=fasttext_data_train,
    data_test=fasttext_data_test,
    name_model='models/fasttext_lstm_cnn'
)

Эпоха 1:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6348867627024964
Точность:		 0.7103914736626358
===Модель сохранена===


Эпоха 2:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.6336874436314082
Точность:		 0.7136708341873335


Эпоха 3:   0%|          | 0/440 [00:00<?, ?it/s]

...

F1-score (weighted):	 0.7477211616684334
Точность:		 0.7622463619594179


Эпоха 29:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7462496885021993
Точность:		 0.7587620414019266


Эпоха 30:   0%|          | 0/440 [00:00<?, ?it/s]

F1-score (weighted):	 0.7446295241601129
Точность:		 0.757532281205165


### Обучение CNN и LSTM моделей для ансамбля

5 моделей CNN и 5 моделей LSTM

In [11]:
# w2v
for i in tqdm(range(10)):
    
    name = f'models/w2v_cnn_and_lstm_ensemble_{i + 1}'
    
    if i < 5:
        train(
            model_class=CNN,
            hyperparameters=best_hyperparameters['w2v_CNN'].copy(),
            data_train=w2v_data_train,
            data_test=w2v_data_test,
            indicators_switch=False,
            name_model=name
        )
    else:
        train(
            model_class=LSTM,
            hyperparameters=best_hyperparameters['w2v_LSTM'].copy(),
            data_train=w2v_data_train,
            data_test=w2v_data_test,
            indicators_switch=False,
            name_model=name
        )

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
# fasttext
for i in tqdm(range(10)):
    
    name = f'models/fasttext_cnn_and_lstm_ensemble_{i + 1}'
    
    if i < 5:
        train(
            model_class=CNN,
            hyperparameters=best_hyperparameters['fasttext_CNN'].copy(),
            data_train=fasttext_data_train,
            data_test=fasttext_data_test,
            indicators_switch=False,
            name_model=name
        )
    else:
        train(
            model_class=LSTM,
            hyperparameters=best_hyperparameters['fasttext_LSTM'].copy(),
            data_train=fasttext_data_train,
            data_test=fasttext_data_test,
            indicators_switch=False,
            name_model=name
        )

  0%|          | 0/10 [00:00<?, ?it/s]

### Обучение LSTM_CNN моделей для ансамбля

10 LSTM_CNN моделей

In [13]:
# w2v
for i in tqdm(range(10)):
    
    name = f'models/w2v_ensemble_of_lstm_cnn_{i + 1}'
    
    train(
        model_class=LSTM_CNN,
        hyperparameters=best_hyperparameters['w2v_LSTM_CNN'].copy(),
        data_train=w2v_data_train,
        data_test=w2v_data_test,
        indicators_switch=False,
        name_model=name
    )

  0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
# fasttext
for i in tqdm(range(10)):
    
    name = f'models/fasttext_ensemble_of_lstm_cnn_{i + 1}'
    
    train(
        model_class=LSTM_CNN,
        hyperparameters=best_hyperparameters['fasttext_LSTM_CNN'].copy(),
        data_train=fasttext_data_train,
        data_test=fasttext_data_test,
        indicators_switch=False,
        name_model=name
    )


  0%|          | 0/10 [00:00<?, ?it/s]

## Результаты

Были обучены и сохранены модели CNN, LSTM, LSTM_CNN и их комбинации для ансамблей на текстах, обработанных Word2Vec и FastText.